In [2]:
import pandas as pd
from chembl_structure_pipeline import standardizer
from chembl_structure_pipeline import checker
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import MolStandardize

In [4]:
df = pd.read_csv("../../Cell_Painting_Median_Features_Median_doses.csv")

In [13]:
df.iloc[1]["InChICode_standardised"]

'InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(3)12(16)18)13-8(14-10)6-19-4/h7H,5-6H2,1-4H3,(H,13,14)'

In [6]:
!pwd

/dgx1nas1/storage/data/seal/The_Seal_Files/The_Seal_dataset/Generate_data


In [7]:
df.InChICode_standardised.nunique()

30404

In [8]:
def standardize_chembl(InChICode_standardised):
    
    try:
        # Read SMILES and convert it to RDKit mol object
        mol = Chem.MolFromInchi(InChICode_standardised)

        desalter = MolStandardize.fragment.LargestFragmentChooser()
        std1_mol = standardizer.standardize_mol(mol)
        desalt_mol = desalter.choose(std1_mol)
        std2_mol = standardizer.standardize_mol(desalt_mol)
        smiles_standardized = Chem.MolToSmiles(std2_mol)

        return smiles_standardized
    
    except:
        return "Cannot_do"

In [9]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=80)

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [10]:
import pandas as pd 
from tqdm import tqdm 
import time 

RDLogger.DisableLog('rdApp.info')  
tqdm.pandas()

In [15]:
df["Standardized_SMILES_chembl"] = df["InChICode_standardised"].parallel_apply(standardize_chembl)

[23:05:10] WARNING: not removing hydrogen atom without neighbors
[23:05:10] WARNING: not removing hydrogen atom without neighbors
[23:05:10] WARNING: not removing hydrogen atom without neighbors
[23:05:10] WARNING: not removing hydrogen atom without neighbors


In [16]:
RDLogger.DisableLog('rdApp.*')  
def smiles_to_inchi(smiles):
    
    try:
        # Read SMILES and convert it to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)
        inchi_standardized = Chem.MolToInchi(mol)

        return inchi_standardized
    
    except:
        return "Cannot_do"

In [17]:
df["Standardized_InChI_chembl"] = df["Standardized_SMILES_chembl"].parallel_apply(smiles_to_inchi)

In [18]:
df

,InChICode_standardised,index,Metadata_mmoles_per_liter,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0,Standardized_SMILES_chembl,Standardized_InChI_chembl
0,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,960.0,5.000000,1728.953125,-1.375000,-25.445312,-0.067918,-0.062396,0.0,0.026443,...,0.222859,0.280258,0.192997,0.144917,0.168085,-0.332871,-0.314733,-0.313477,COC(=O)Nc1nc2ccc(SC(C)C)cc2[nH]1,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...
1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,1036.0,5.000000,-272.074219,11.464844,-1.628906,-0.001808,-0.006674,0.0,-0.001405,...,-0.016903,-0.017773,-0.028029,-0.038308,-0.041455,-0.070389,-0.067789,-0.071673,COCc1nc2c(=O)n(C)c(=O)n(CC(C)C)c2[nH]1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...
2,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,4942.5,4.973595,101.958984,15.673828,-2.314453,0.010687,0.013397,0.0,-0.003746,...,0.035703,0.016875,-0.039304,-0.070028,-0.062892,-0.015647,-0.038908,-0.026464,CNC[C@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...
3,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,4944.5,4.912934,-109.708984,-10.203125,1.718750,0.025025,0.014627,0.0,-0.007164,...,0.031759,0.037029,-0.051411,-0.083061,-0.071562,-0.024598,-0.052021,-0.043327,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...
4,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,4949.5,5.012874,-234.222656,8.873047,3.308594,0.010192,0.005251,0.0,-0.003173,...,-0.030955,-0.037623,-0.009460,0.004604,-0.010465,-0.031869,-0.025910,-0.014907,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30399,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,30588.0,5.000000,-226.027344,-18.847656,-0.511719,-0.005827,0.004864,0.0,0.003590,...,-0.123664,-0.127474,-0.069665,-0.056972,-0.044186,-0.019668,-0.091541,-0.062515,NNC(S)=Nc1cccc(-c2nnc(S)o2)c1,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...
30400,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,30589.0,5.000000,254.769531,18.886719,-3.589844,0.027814,0.016943,0.0,-0.000781,...,-0.018155,-0.017759,-0.118369,-0.046322,-0.023926,0.033498,0.051214,0.016060,OCc1c[nH]c2ccccc12,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...
30401,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,30590.0,5.000000,238.574219,-0.699219,6.625000,0.005228,0.008800,0.0,0.002336,...,-0.107077,-0.138399,0.168811,0.216703,0.195851,-0.063656,-0.042877,-0.071681,Nc1cccc(/C=C/C(=O)O)c1,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...
30402,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,30591.0,5.000000,-238.945312,5.476562,0.089844,0.018032,0.012711,0.0,-0.004635,...,-0.095379,-0.102060,0.023423,0.049321,0.041996,0.023470,0.058959,0.065934,CC(=O)CSc1ccc(C(=O)O)cn1,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...


In [19]:
RDLogger.DisableLog('rdApp.*')  
def smiles_to_inchikey(smiles):
    
    try:
        # Read SMILES and convert it to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)
        inchikey_standardized = Chem.MolToInchiKey(mol)

        return inchikey_standardized
    
    except:
        return "Cannot_do"

In [20]:
df["Standardized_InChIKey_chembl"] = df["Standardized_SMILES_chembl"].parallel_apply(smiles_to_inchikey)

In [23]:
df.InChICode_standardised == df.Standardized_InChI_chembl

0        True
1        True
2        True
3        True
4        True
         ... 
30399    True
30400    True
30401    True
30402    True
30403    True
Length: 30404, dtype: bool

In [26]:
df = df[["InChICode_standardised", "Standardized_SMILES_chembl", "Standardized_InChI_chembl", "Standardized_InChIKey_chembl"]]
df

,InChICode_standardised,Standardized_SMILES_chembl,Standardized_InChI_chembl,Standardized_InChIKey_chembl
0,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,COC(=O)Nc1nc2ccc(SC(C)C)cc2[nH]1,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,SXJIZQPZESTWLD-UHFFFAOYSA-N
1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,COCc1nc2c(=O)n(C)c(=O)n(CC(C)C)c2[nH]1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,NBLBCGUCPBXKOV-UHFFFAOYSA-N
2,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-GZBFAFLISA-N
3,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-IPYPFGDCSA-N
4,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-NFAWXSAZSA-N
...,...,...,...,...
30399,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,NNC(S)=Nc1cccc(-c2nnc(S)o2)c1,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,LHXWPDUWZAYBJV-UHFFFAOYSA-N
30400,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,OCc1c[nH]c2ccccc12,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,IVYPNXXAYMYVSP-UHFFFAOYSA-N
30401,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,Nc1cccc(/C=C/C(=O)O)c1,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,JNXMJSYJCFTLJB-SNAWJCMRSA-N
30402,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,CC(=O)CSc1ccc(C(=O)O)cn1,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,LRBIKSYRCFHKBN-UHFFFAOYSA-N


In [27]:
df.Standardized_InChIKey_chembl.nunique()

30404

In [28]:
df.to_csv("Bray_compounds_processed_chembl_identifiers.csv.gz", index=False, compression="gzip")

In [29]:
import pandas as pd
df = pd.read_csv("Bray_compounds_processed_chembl_identifiers.csv.gz", compression="gzip")
df

,InChICode_standardised,Standardized_SMILES_chembl,Standardized_InChI_chembl,Standardized_InChIKey_chembl
0,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,COC(=O)Nc1nc2ccc(SC(C)C)cc2[nH]1,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,SXJIZQPZESTWLD-UHFFFAOYSA-N
1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,COCc1nc2c(=O)n(C)c(=O)n(CC(C)C)c2[nH]1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,NBLBCGUCPBXKOV-UHFFFAOYSA-N
2,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-GZBFAFLISA-N
3,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-IPYPFGDCSA-N
4,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-NFAWXSAZSA-N
...,...,...,...,...
30399,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,NNC(S)=Nc1cccc(-c2nnc(S)o2)c1,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,LHXWPDUWZAYBJV-UHFFFAOYSA-N
30400,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,OCc1c[nH]c2ccccc12,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,IVYPNXXAYMYVSP-UHFFFAOYSA-N
30401,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,Nc1cccc(/C=C/C(=O)O)c1,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,JNXMJSYJCFTLJB-SNAWJCMRSA-N
30402,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,CC(=O)CSc1ccc(C(=O)O)cn1,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,LRBIKSYRCFHKBN-UHFFFAOYSA-N


In [30]:
from chembl_webresource_client.new_client import new_client

# Set up the ChEMBL client
chembl_client = new_client

In [31]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=80)

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
def smiles_to_chembl_id(smiles):
    try:
        molecule = chembl_client.molecule
        result = molecule.filter(smiles=smiles).only("molecule_chembl_id")
        return result[0]["molecule_chembl_id"] if result else None
    except Exception as e:
        print(f"Error converting SMILES to ChEMBL ID for {smiles}: {e}")
        return "Cannot_do"

In [39]:
def inchikey_to_chembl_id(inchikey):
    result = None
    
    try:
        molecule = chembl_client.molecule
        result = molecule.filter(molecule_structures__standard_inchi_key=inchikey).only("molecule_chembl_id")
        return result[0]["molecule_chembl_id"] if result else None
    except Exception as e:
        print(f"Error converting InChI key to ChEMBL ID for {inchikey}: {e}")
        return "Cannot_do"


In [34]:
# Apply the function to each row and create a new 'ChEMBL_ID' column
df['ChEMBL_ID'] = df['Standardized_InChIKey_chembl'].parallel_apply(inchikey_to_chembl_id)
df

,InChICode_standardised,Standardized_SMILES_chembl,Standardized_InChI_chembl,Standardized_InChIKey_chembl,ChEMBL_ID
0,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,COC(=O)Nc1nc2ccc(SC(C)C)cc2[nH]1,InChI=1S/C12H15N3O2S/c1-7(2)18-8-4-5-9-10(6-8)...,SXJIZQPZESTWLD-UHFFFAOYSA-N,CHEMBL1354363
1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,COCc1nc2c(=O)n(C)c(=O)n(CC(C)C)c2[nH]1,InChI=1S/C12H18N4O3/c1-7(2)5-16-10-9(11(17)15(...,NBLBCGUCPBXKOV-UHFFFAOYSA-N,CHEMBL307639
2,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-GZBFAFLISA-N,None
3,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-IPYPFGDCSA-N,None
4,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,CNC[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C,InChI=1S/C16H29N5O3/c1-12-9-20(13(2)10-22)16(2...,FOLOUAULJSJVDY-NFAWXSAZSA-N,CHEMBL3187280
...,...,...,...,...,...
30399,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,NNC(S)=Nc1cccc(-c2nnc(S)o2)c1,InChI=1S/C9H9N5OS2/c10-12-8(16)11-6-3-1-2-5(4-...,LHXWPDUWZAYBJV-UHFFFAOYSA-N,CHEMBL1432353
30400,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,OCc1c[nH]c2ccccc12,InChI=1S/C9H9NO/c11-6-7-5-10-9-4-2-1-3-8(7)9/h...,IVYPNXXAYMYVSP-UHFFFAOYSA-N,CHEMBL155625
30401,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,Nc1cccc(/C=C/C(=O)O)c1,InChI=1S/C9H9NO2/c10-8-3-1-2-7(6-8)4-5-9(11)12...,JNXMJSYJCFTLJB-SNAWJCMRSA-N,CHEMBL1379670
30402,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,CC(=O)CSc1ccc(C(=O)O)cn1,InChI=1S/C9H9NO3S/c1-6(11)5-14-8-3-2-7(4-10-8)...,LRBIKSYRCFHKBN-UHFFFAOYSA-N,CHEMBL1305263


In [37]:
df.ChEMBL_ID.nunique()

12452

In [38]:
df.to_csv("Bray_compounds_processed_chembl_identifiers.csv.gz", index=False, compression="gzip")